In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard imports
import numpy as np
import xarray as xr
import tqdm as tqdm

In [3]:
# Import OpenSense modules as submodules
import sys
import os

sys.path.append(os.path.abspath("./pycomlink/"))
sys.path.append(os.path.abspath("./poligrain/src/"))
sys.path.append(os.path.abspath("./mergeplg/src/"))

import pycomlink as pycml 
import poligrain as plg
import mergeplg 

# OpenMRG

In [9]:
# OpenMRG
ds_rad = xr.open_dataset("data/andersson_2022_OpenMRG/radar/openmrg_rad.nc")                    
ds_cmls = xr.open_dataset("data/processed_cml_OpenMRG.nc").R_acc                                  

# Adjust rainfall fields

In [10]:
os.makedirs('data/adjusted_fields', exist_ok=True)
months = ['2015-06', '2015-07', '2015-08']

In [6]:
# additive IDW
nnear = 12 
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="additive",
        nnear=nnear,
        range_checks={'diff_check':10},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_IDW": xr.concat(rainfall, dim="time") })
data['rainfall_add_IDW']=xr.where(
    np.isnan(data['rainfall_add_IDW']), 0, data['rainfall_add_IDW']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_add_IDW.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:09<00:00, 78.58it/s]


month: 2015-07


100%|██████████| 744/744 [00:08<00:00, 85.52it/s]


month: 2015-08


100%|██████████| 744/744 [00:09<00:00, 80.58it/s]


In [7]:
# multiplicative IDW
nnear = 12 
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="multiplicative",
        nnear=nnear,
        range_checks={'ratio_check':(0.1,15)},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_IDW": xr.concat(rainfall, dim="time") })
data['rainfall_mul_IDW']=xr.where(
    np.isnan(data['rainfall_mul_IDW']), 0, data['rainfall_mul_IDW']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_mul_IDW.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:08<00:00, 80.19it/s]


month: 2015-07


100%|██████████| 744/744 [00:09<00:00, 79.87it/s]


month: 2015-08


100%|██████████| 744/744 [00:09<00:00, 78.04it/s]


In [8]:
# additive POINT BLOCK KRIGING
variogram_parameters = {"sill": 1.0, "range": 30000, "nugget": 0.1}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={'diff_check':10},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_pBK": xr.concat(rainfall, dim="time") })
data['rainfall_add_pBK']=xr.where(
    np.isnan(data['rainfall_add_pBK']), 0, data['rainfall_add_pBK']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_add_pBK.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:17<00:00, 40.62it/s] 


month: 2015-07


100%|██████████| 744/744 [00:21<00:00, 34.71it/s] 


month: 2015-08


100%|██████████| 744/744 [00:18<00:00, 40.84it/s] 


In [9]:
# multiplicative POINT BLOCK KRIGING
variogram_parameters = {"sill": 1.0, "range": 30000, "nugget": 0.1}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={'ratio_check':(0.1,15)},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_pBK": xr.concat(rainfall, dim="time") })
data['rainfall_mul_pBK']=xr.where(
    np.isnan(data['rainfall_mul_pBK']), 0, data['rainfall_mul_pBK']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_mul_pBK.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:14<00:00, 48.58it/s] 


month: 2015-07


100%|██████████| 744/744 [00:17<00:00, 41.53it/s] 


month: 2015-08


100%|██████████| 744/744 [00:14<00:00, 49.82it/s] 


In [10]:
# additive LINE BLOCK KRIGING
variogram_parameters = {"sill": 1.0, "range": 30000, "nugget": 0.1}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={'diff_check':10},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_lBK": xr.concat(rainfall, dim="time") })
data['rainfall_add_lBK']=xr.where(
    np.isnan(data['rainfall_add_lBK']), 0, data['rainfall_add_lBK']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_add_lBK.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:17<00:00, 41.80it/s] 


month: 2015-07


100%|██████████| 744/744 [00:21<00:00, 35.32it/s] 


month: 2015-08


100%|██████████| 744/744 [00:17<00:00, 41.37it/s] 


In [11]:
# multiplicative LINE BLOCK KRIGING
variogram_parameters = {"sill": 1.0, "range": 30000, "nugget": 0.1}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={'ratio_check':(0.1,15)},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_lBK": xr.concat(rainfall, dim="time") })
data['rainfall_mul_lBK']=xr.where(
    np.isnan(data['rainfall_mul_lBK']), 0, data['rainfall_mul_lBK']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_mul_lBK.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:15<00:00, 46.88it/s] 


month: 2015-07


100%|██████████| 744/744 [00:18<00:00, 40.00it/s] 


month: 2015-08


100%|██████████| 744/744 [00:15<00:00, 48.91it/s] 


In [11]:
# KED point
variogram_parameters = {"sill": 1.0, "range": 30000, "nugget": 0.1}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=False,
        range_checks={'diff_check':10},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_ked": xr.concat(rainfall, dim="time") })
data['rainfall_ked']=xr.where(
    np.isnan(data['rainfall_ked']), 0, data['rainfall_ked']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_pked.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:18<00:00, 38.80it/s] 


month: 2015-07


100%|██████████| 744/744 [00:21<00:00, 35.23it/s] 


month: 2015-08


100%|██████████| 744/744 [00:17<00:00, 42.51it/s] 


In [12]:
# KED line
variogram_parameters = {"sill": 1.0, "range": 30000, "nugget": 0.1}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=True,
        range_checks={'diff_check':10},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_ked": xr.concat(rainfall, dim="time") })
data['rainfall_ked']=xr.where(
    np.isnan(data['rainfall_ked']), 0, data['rainfall_ked']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_lked.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:17<00:00, 40.10it/s] 


month: 2015-07


100%|██████████| 744/744 [00:22<00:00, 32.77it/s] 


month: 2015-08


100%|██████████| 744/744 [00:17<00:00, 41.77it/s] 


# OpenRainER

In [5]:
# OpenRainER
ds_rad = xr.open_dataset("data/covi_2024_OpenRainER/openrainer_radar.nc")         
ds_cmls = xr.open_dataset("data/processed_cml_OpenRainER.nc").R_acc                     

# Adjust rainfall fields

In [6]:
os.makedirs('data/adjusted_fields', exist_ok=True)
months = ['2022-06', '2022-07', '2022-08']

In [15]:
# additive IDW
nnear = 12 
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="additive",
        nnear=nnear,
        range_checks={'diff_check':10},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_IDW": xr.concat(rainfall, dim="time") })
data['rainfall_add_IDW']=xr.where(
    np.isnan(data['rainfall_add_IDW']), 0, data['rainfall_add_IDW']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_add_IDW.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [00:46<00:00, 15.50it/s]


month: 2022-07


100%|██████████| 744/744 [00:47<00:00, 15.74it/s]


month: 2022-08


100%|██████████| 744/744 [00:52<00:00, 14.21it/s]


In [17]:
# multiplicative IDW
nnear = 12 
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="multiplicative",
        nnear=nnear,
        range_checks={'ratio_check':(0.1,15)},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_IDW": xr.concat(rainfall, dim="time") })
data['rainfall_mul_IDW']=xr.where(
    np.isnan(data['rainfall_mul_IDW']), 0, data['rainfall_mul_IDW']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_mul_IDW.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [00:35<00:00, 20.08it/s]


month: 2022-07


100%|██████████| 744/744 [00:36<00:00, 20.63it/s]


month: 2022-08


100%|██████████| 744/744 [00:37<00:00, 19.86it/s]


In [18]:
# additive POINT BLOCK KRIGING
variogram_parameters = {"sill": 1.0, "range": 30000, "nugget": 0.1}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={'diff_check':10},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_pBK": xr.concat(rainfall, dim="time") })
data['rainfall_add_pBK']=xr.where(
    np.isnan(data['rainfall_add_pBK']), 0, data['rainfall_add_pBK']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_add_pBK.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [08:28<00:00,  1.41it/s]


month: 2022-07


100%|██████████| 744/744 [02:12<00:00,  5.60it/s]


month: 2022-08


100%|██████████| 744/744 [06:28<00:00,  1.91it/s]


In [19]:
# multiplicative POINT BLOCK KRIGING
variogram_parameters = {"sill": 1.0, "range": 30000, "nugget": 0.1}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={'ratio_check':(0.1,15)},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_pBK": xr.concat(rainfall, dim="time") })
data['rainfall_mul_pBK']=xr.where(
    np.isnan(data['rainfall_mul_pBK']), 0, data['rainfall_mul_pBK']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_mul_pBK.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [04:24<00:00,  2.72it/s]


month: 2022-07


100%|██████████| 744/744 [01:20<00:00,  9.28it/s] 


month: 2022-08


100%|██████████| 744/744 [04:15<00:00,  2.91it/s]


In [20]:
# additive LINE BLOCK KRIGING
variogram_parameters = {"sill": 1.0, "range": 30000, "nugget": 0.1}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={'diff_check':10},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_lBK": xr.concat(rainfall, dim="time") })
data['rainfall_add_lBK']=xr.where(
    np.isnan(data['rainfall_add_lBK']), 0, data['rainfall_add_lBK']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_add_lBK.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [08:06<00:00,  1.48it/s]


month: 2022-07


100%|██████████| 744/744 [02:16<00:00,  5.44it/s]


month: 2022-08


100%|██████████| 744/744 [07:51<00:00,  1.58it/s]


In [21]:
# multiplicative LINE BLOCK KRIGING
variogram_parameters = {"sill": 1.0, "range": 30000, "nugget": 0.1}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={'ratio_check':(0.1,15)},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_lBK": xr.concat(rainfall, dim="time") })
data['rainfall_mul_lBK']=xr.where(
    np.isnan(data['rainfall_mul_lBK']), 0, data['rainfall_mul_lBK']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_mul_lBK.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [04:40<00:00,  2.56it/s]


month: 2022-07


100%|██████████| 744/744 [01:30<00:00,  8.21it/s] 


month: 2022-08


100%|██████████| 744/744 [04:26<00:00,  2.79it/s]


In [7]:
# KED point
variogram_parameters = {"sill": 1.0, "range": 30000, "nugget": 0.1}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=False,
        range_checks={'diff_check':10},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_ked": xr.concat(rainfall, dim="time") })
data['rainfall_ked']=xr.where(
    np.isnan(data['rainfall_ked']), 0, data['rainfall_ked']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_pked.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [07:27<00:00,  1.61it/s]


month: 2022-07


100%|██████████| 744/744 [02:00<00:00,  6.20it/s]


month: 2022-08


100%|██████████| 744/744 [05:53<00:00,  2.11it/s]


In [8]:
# KED line
variogram_parameters = {"sill": 1.0, "range": 30000, "nugget": 0.1}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=True,
        range_checks={'diff_check':10},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_ked": xr.concat(rainfall, dim="time") })
data['rainfall_ked']=xr.where(
    np.isnan(data['rainfall_ked']), 0, data['rainfall_ked']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_lked.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [07:49<00:00,  1.53it/s]


month: 2022-07


100%|██████████| 744/744 [01:50<00:00,  6.76it/s]


month: 2022-08


100%|██████████| 744/744 [05:31<00:00,  2.25it/s]
